# Highcode Example: Experimentation based on SimuCredit Data

## 0) Setting up Modeva

In [ ]:
## =============================================================
## Install or update packages(recommended to run in Terminal)
## =============================================================
!pip show modeva
# !pip uninstall modeva
#!pip install modeva

## 1) Data Modules

- Data Loading
- Data Summary
- Exploratory Data Analysis (EDA)
- Data Processing

In [ ]:
## Create an instance of DataSet class
from modeva import DataSet
ds = DataSet()

In [ ]:
## ----------------------------------------------------------------
## Data Loading: a) built-in data; b) user data
## ----------------------------------------------------------------

## a) Load built-in data: "BikeSharing", "CaliforniaHousing", "SimuCredit", "TaiwanCredit"
ds.load("SimuCredit")
ds

In [ ]:
## b) Load user data as pandas dataframe

# import pandas as pd
# data = pd.read_csv("SimuCredit.csv")
# ds = DataSet()
# ds.load_dataframe(data)

In [ ]:
## ----------------------------------------------------------------
## Data Summary: descriptive statistics
## ----------------------------------------------------------------

result = ds.summary()

In [ ]:
## Overall data summary
result.table["summary"]

In [ ]:
## Summary of numerical features
result.table["numerical"]

In [ ]:
## Summary of categorical features
result.table["categorical"]

In [ ]:
## ----------------------------------------------------------------
## Exploratory Data Analysis (EDA): 1d, 2d, 3d, correlation, pca, umap
## ----------------------------------------------------------------

result = ds.eda_1d(feature="Balance", plot_type="density")
result.plot(figsize=(5, 4))
result = ds.eda_1d(feature="Balance", plot_type="histogram")
result.plot(figsize=(5, 4))

In [ ]:
## 2D plots: pair of numerical features
result = ds.eda_2d(feature_x="Balance", feature_y="Mortgage", feature_color="Race",
                   sample_size=300, smoother_order=None)
result.plot(figsize=(5, 4))

In [ ]:
## 2D plots: pair of categorical features
result = ds.eda_2d(feature_x="Race", feature_y="Gender")
result.plot(figsize=(5, 4))

In [ ]:
## 2D plots: numerical and categorical features
result = ds.eda_2d(feature_x="Balance", feature_y="Gender")
result.plot(figsize=(5, 4))

In [ ]:
## 3D Scatter Plot
result = ds.eda_3d(feature_x="Mortgage", feature_y="Balance", feature_z="Utilization",
                   feature_color="Status", sample_size=300)
result.plot(figsize=(6, 5))

In [ ]:
## Correlation Heatmap
result = ds.eda_correlation(features=('Mortgage',
                                      'Balance',
                                      'Amount Past Due',
                                      'Delinquency',
                                      'Inquiry',
                                      'Open Trade',
                                      'Utilization'), sample_size=10000)
result.plot(figsize=(5, 5))

In [ ]:
## PCA - Dimension Reduction
result = ds.eda_pca(features=('Mortgage',
                              'Balance',
                              'Amount Past Due',
                              'Delinquency',
                              'Inquiry',
                              'Open Trade',
                              'Utilization'), n_components=5)
result.plot(figsize=(5, 6))

In [ ]:
## ----------------------------------------------------------------
## Data Preprocessing and Feature Engineering
## 
##    ds.impute_missing: missing value imputation
##    ds.scale_numerical: scaling, standardization of numerical features
##    ds.encode_categorical: one-hot encoding or ordinal encoding for categorical features
##    ds.bin_numerical: binning numerical features into discrete bins
##
## Upon calling these functions, no results will be returned. 
## Data processing will be executed by running ds.preprocess(). 
## To reset preprocessing steps, run ds.reset_preprocess().
## ----------------------------------------------------------------

## First of all, reset all preprocessing steps
ds.reset_preprocess()

## a) data imputation
ds.impute_missing()

## b) scaling for numerical features
ds.scale_numerical(method="minmax")

## c) encoding for numerical features
ds.encode_categorical(features=("Race", "Gender"), method="ordinal")

## d) binning for numerical features
ds.bin_numerical(features=("Amount Past Due", ), bins=10, method="uniform")

## e) execute all preprocessing steps
ds.preprocess()

## Display the preprocessed data
ds.data

In [ ]:
## Compare to the raw data
ds.raw_data

In [ ]:
## ----------------------------------------------------------------
## Other Data Processing Functions
## 
##    ds.set_active_features (ds.set_inactive_features): set some features to be active or inactive
##    ds.set_target: set the target feature
##    ds.set_sample_weight: set the sample_weight feature
##    ds.set_feature_type: change the feature type
##    ds.set_task_type: change task type, including "Regression" and "Classification"
##    ds.set_active_samples (set_inactive_samples): set active samples, used for subsampling or outlier removal
##    ds.set_random_split: automatically set train test split (purly random, on (subsampled if exist) "main" data)
##    ds.set_train_idx: manually set training set index
##    ds.set_test_idx: manually set testing set index
## ----------------------------------------------------------------

## a) set inactive features
ds.set_inactive_features(features=['Race', 'Gender'])

## b) set target feature
ds.set_target(feature="Status")

## c) set task type
ds.set_task_type('Classification')

## d) change feature types
ds.set_random_split(test_ratio=0.33)

## 2) Model Modules

- Built-in interpretable models: GLM, DecisionTree, GBDT, RandomForest, XGB, LGBM, CatBoost, GAMINet, ReLuDNN, GLMTree, GLMTreeBoost, NeuralTree, MOE
- Model Training
- Model Tuning
- Model Wrapping
- Model Interpretability
- Model Post-hoc Explainability

In [ ]:
## ----------------------------------------------------------------
## Model Training: e.g. LGBM
## ----------------------------------------------------------------

from modeva.models import MoLGBMClassifier
model = MoLGBMClassifier(name="LGBM", max_depth=2, n_estimators=100, verbose=-1)
model.fit(ds.train_x, ds.train_y.ravel())

In [ ]:
## ----------------------------------------------------------------
## Model Tuning: e.g. Random Search
## ----------------------------------------------------------------

from modeva.models.tune import ModelTuneRandomSearch
from scipy.stats import loguniform 
hyperspace = dict(eta=loguniform(a=2**(-5), b=1.0),
                  max_depth=[1, 2, 3, 4, 5])
hpo = ModelTuneRandomSearch(dataset=ds,
                            model=MoLGBMClassifier(verbose=-1))
result = hpo.run(param_distributions=hyperspace,
                 n_iter=10,
                 metric="AUC",
                 cv=5,
                 random_state=0)
result.table

In [ ]:
## ----------------------------------------------------------------
## Refit the model using selected hyperparameter
## ----------------------------------------------------------------
model_tuned = MoLGBMClassifier(**result.value["params"][5],
                               name="LGMB-Tuned",
                               verbose=-1)
model_tuned.fit(ds.train_x, ds.train_y)
model_tuned

In [ ]:
## ----------------------------------------------------------------
## Model Wrapping: e.g. pre-trained Sklearn-style model
## ----------------------------------------------------------------

from xgboost import XGBClassifier
from modeva.models import MoSKLearnRegressor, MoSKLearnClassifier
model_sk = MoSKLearnClassifier(estimator=XGBClassifier(), name="WrappedXGB") 
model_sk.fit(ds.train_x, ds.train_y.ravel())

In [ ]:
## ----------------------------------------------------------------
## Model Interpretability: 
##    fs.interpret_fi: feature importance
##    fs.interpret_ei: effect importance
##    fs.interpret_local_fi: local feature importance
##    fs.interpret_local_ei: local effect importance
##    fs.interpret_effects: global effect plot
##
## Post-hoc Explainability
##    fs.explain_pfi: permutation feature importance
##    fs.explain_hstatistic: H-statistic for each pair of features
##    fs.explain_pdp: 1D and 2D PDP
##    fs.explain_ale: 1D and 2D ALE
##    fs.explain_lime: LIME for local explanation
##    fs.explain_shap: SHAP for local explanation
## ----------------------------------------------------------------

## Create a factsheet that bundles dataset and model
from modeva import FactSheet
fs = FactSheet(ds, model)

In [ ]:
## Global feature importance and effect importance
result = fs.interpret_fi()
result.plot(figsize=(6, 4))
result = fs.interpret_ei()
result.plot(figsize=(6, 4))

In [ ]:
## Global effect plots
result = fs.interpret_effects(features="Delinquency")
result.plot(figsize=(6, 4))
result = fs.interpret_effects(features=("Delinquency", "Utilization"))
result.plot(figsize=(6, 5))

In [ ]:
## Local feature importance and effect importance
result = fs.interpret_local_fi(dataset='test', sample_index=0, centered=True)
result.plot(figsize=(6, 4))
result = fs.interpret_local_ei(dataset='test', sample_index=0)
result.plot(figsize=(6, 4))

In [ ]:
## Post-hoc permutation feature importance
result = fs.explain_pfi()
result.plot(figsize=(6, 4))

In [ ]:
## Post-hoc H-statistic
result = fs.explain_hstatistic(sample_size=1000, grid_resolution=10)
result.plot(figsize=(6, 5))

In [ ]:
## Post-hoc partial dependence plots
result = fs.explain_pdp(features="Delinquency")
result.plot(figsize=(6, 4))
result = fs.explain_pdp(features=("Delinquency", "Utilization"))
result.plot(figsize=(6, 5))

In [ ]:
## Post-hoc accumulated local effects
result = fs.explain_ale(features=("Delinquency", "Utilization"), dataset="train")
result.plot(figsize=(6, 5))

In [ ]:
## Post-hoc local explainability (LIME and SHAP)
result = fs.explain_lime(dataset="test", sample_index=0, centered=False)
result.plot(figsize=(6.5, 4))
result = fs.explain_shap(dataset="test", sample_index=0)
result.plot(figsize=(6.5, 4))

## 3) Test Modules

- Tests for a single model
- Slicing diagnostics
- Model benchmarking
- Fairness Tests

In [ ]:
## ----------------------------------------------------------------
## Tests for a single model:
##    fs.diagnose_accuracy_table
##    fs.diagnose_residual_analysis
##    fs.diagnose_reliability
##    fs.diagnose_robustness
##    fs.diagnose_resilience
## ----------------------------------------------------------------

## Performance metrics
result = fs.diagnose_accuracy_table(train_dataset="train", test_dataset="test", metric=None)
result.table

In [ ]:
## Residual analysis
result = fs.diagnose_residual_analysis(features="Balance", dataset="test")
result.plot(figsize=(6, 4))

In [ ]:
## Reliability (Venn-amber prediction for binary classification; Conformal prediction for regression)
result = fs.diagnose_reliability(train_dataset="test", test_dataset="test",
                                  test_size=0.5, random_state=0)
result.plot(figsize=(6, 4))

In [ ]:
## Robustness 
result = fs.diagnose_robustness(dataset="test", perturb_features=None, 
                                noise_levels=(0.2, 0.4, 0.6, 0.8), metric="ACC")
result.plot(figsize=(6, 4))

In [ ]:
# Resilience
result = fs.diagnose_resilience(method="worst-sample", metric="AUC")
result.plot(figsize=(6, 4))

In [ ]:
## ----------------------------------------------------------------
## Slicing-based tests:
##    fs.diagnose_slicing_fi
##    fs.diagnose_slicing_accuracy
##    fs.diagnose_slicing_overfit
##    fs.diagnose_slicing_reliability
##    fs.diagnose_slicing_robustness
##    fs.diagnose_slicing_fairness
## ----------------------------------------------------------------

result = fs.diagnose_residual_fi(method="uniform")
result.plot(figsize=(6, 4))

In [ ]:
result = fs.diagnose_slicing_accuracy(features=(("Mortgage",), ("Utilization", ), ), metric="AUC",
                                      method="quantile", threshold=None)
result.table

In [ ]:
result = fs.diagnose_slicing_accuracy(features=("Mortgage", "Utilization"), method="uniform", bins=10,
                                      metric="ACC", threshold=0.5)
result.plot(figsize=(6, 5))

In [ ]:
result = fs.diagnose_slicing_overfit(train_dataset="train", test_dataset="test",
                                     features="Mortgage", metric="AUC", threshold=None)
result.plot(figsize=(6, 5))

In [ ]:
## ----------------------------------------------------------------
## Model bencharmking/comparison tests:
##    fsc.compare_accuracy_table
##    fsc.compare_robustness
##    fsc.compare_reliability
##    fsc.compare_resilience
## ----------------------------------------------------------------

## create FactSheet that bundles dataset and multiple models
fsc = FactSheet(ds, models=[model, model_sk])

In [ ]:
result = fsc.compare_accuracy_table(train_dataset="train", test_dataset="test", 
                                    metric=("AUC", "LogLoss"))
result.plot(figsize=(5, 4))

In [ ]:
result = fsc.compare_robustness(perturb_features=("Balance", "Mortgage", ), noise_levels=(0.2, 0.4, 0.6, 0.8), 
                                perturb_method="quantile", metric="AUC")
result.plot(figsize=(6, 4))

In [ ]:
result = fsc.compare_slicing_accuracy(features="Utilization", method="uniform", bins=5, metric="AUC")
result.plot(figsize=(6, 5))

In [ ]:
result = fsc.compare_slicing_overfit(features="Utilization", method="uniform", bins=5, metric="AUC")
result.plot(figsize=(6, 5))

In [ ]:
## ----------------------------------------------------------------
## Fairness Tests
##    fs.diagnose_fairness
##    fs.diagnose_slicing_fairness
##    fsc.compare_fairness
##    fsc.compare_slicing_fairness
##    fs.diagnose_mitigate_unfair_thresholding
##    fs.diagnose_mitigate_unfair_binning
## ----------------------------------------------------------------

## set protected data using pd.dataframe (in practice, modeling data and demographic data should be in separated files)
ds.set_protected_data(ds.raw_data[["Race", "Gender"]])

In [ ]:
group_config = {
    "Race": {"feature": "Race",
             "protected": 0.0,
             "reference": 1.0},
    "Gender": {"feature": "Gender",
               "protected": 0.0,
               "reference": 1.0}
}

In [ ]:
result = fs.diagnose_fairness(dataset="train",
                              group_config=group_config,
                              favorable_label=1,
                              metric="AIR",
                              threshold=0.8)
result.plot(figsize=(6, 5))

In [ ]:
result = fs.diagnose_slicing_fairness(features="Balance",
                                      dataset="train",
                                      group_config=group_config,
                                      favorable_label=1,
                                      metric="AIR",
                                      threshold=None)
result.plot(figsize=(6, 5))

In [ ]:
result = fsc.compare_fairness(dataset="train",
                              group_config=group_config,
                              favorable_label=1,
                              metric="AIR",
                              threshold=0.8)
result.plot(figsize=(6, 4))

In [ ]:
result = fs.diagnose_mitigate_unfair_thresholding(group_config=group_config,
                                                  favorable_label=1,
                                                  dataset="train",
                                                  metric="PR",
                                                  proba_cutoff=(0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9))
result.plot(figsize=(6, 4))